In [15]:
import io
import base64

import torch
import numpy as np
from faster_whisper import WhisperModel
from faster_whisper.audio import decode_audio
from IPython.display import Audio  

assert torch.cuda.is_available(), "CUDA is not available. Please check your PyTorch installation."

In [2]:
model = WhisperModel("large-v3-turbo", device="cuda", compute_type="float16")

In [10]:
request = {
    "language":"pt",
    "audio_base64":"..."
}

In [11]:
audio_bytes = base64.b64decode(request['audio_base64'])

In [13]:
type(audio_bytes)
isinstance(audio_bytes, bytes)

True

In [ ]:
async def get_transcription(audio_bytes:bytes) -> str:
    """
    Transcribe the audio bytes to text using OpenAI's Whisper API.
    """
    # Assuming you have a function `transcribe_audio` that handles the API call
    audio_np = decode_audio(io.BytesIO(audio_bytes))
    assert isinstance(audio_np, np.ndarray), "Audio data is not in the expected format"
    segments,_ = model.transcribe(audio_np, language='pt')
    transcription = " ".join([segment.text for segment in segments])
    return transcription

In [18]:
t = await get_transcription(audio_bytes)
t

' Olá, Lucas.'